In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import ipywidgets as widgets

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,81.05,46,0,9.48,clear sky
1,1,Broome,US,42.2506,-75.8330,16.02,76,0,8.05,clear sky
2,2,Presidencia Roque Saenz Pena,AR,-26.7852,-60.4388,70.09,38,0,1.43,clear sky
3,3,Beira,MZ,-19.8436,34.8389,75.81,85,16,3.40,few clouds
4,4,Punta Arenas,CL,-53.1500,-70.9167,44.71,49,20,8.05,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)]
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,81.05,46,0,9.48,clear sky
3,3,Beira,MZ,-19.8436,34.8389,75.81,85,16,3.40,few clouds
5,5,Avarua,CK,-21.2078,-159.7750,75.25,88,20,4.61,light rain
12,12,Kapaa,US,22.0752,-159.3190,78.78,64,0,17.27,clear sky
14,14,Atuona,PF,-9.8000,-139.0333,77.61,79,17,13.56,light rain
...,...,...,...,...,...,...,...,...,...,...
687,687,Palompon,PH,11.0508,124.3843,82.00,72,100,5.01,overcast clouds
688,688,Aden,YE,12.7794,45.0367,75.81,71,6,8.66,clear sky
690,690,Ventanas,EC,-1.4500,-79.4667,76.05,90,100,0.40,moderate rain
693,693,Vao,NC,-22.6667,167.4833,80.46,80,99,9.13,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,81.05,46,0,9.48,clear sky
3,3,Beira,MZ,-19.8436,34.8389,75.81,85,16,3.40,few clouds
5,5,Avarua,CK,-21.2078,-159.7750,75.25,88,20,4.61,light rain
12,12,Kapaa,US,22.0752,-159.3190,78.78,64,0,17.27,clear sky
14,14,Atuona,PF,-9.8000,-139.0333,77.61,79,17,13.56,light rain
...,...,...,...,...,...,...,...,...,...,...
687,687,Palompon,PH,11.0508,124.3843,82.00,72,100,5.01,overcast clouds
688,688,Aden,YE,12.7794,45.0367,75.81,71,6,8.66,clear sky
690,690,Ventanas,EC,-1.4500,-79.4667,76.05,90,100,0.40,moderate rain
693,693,Vao,NC,-22.6667,167.4833,80.46,80,99,9.13,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,81.05,clear sky,-33.6500,115.3333,
3,Beira,MZ,75.81,few clouds,-19.8436,34.8389,
5,Avarua,CK,75.25,light rain,-21.2078,-159.7750,
12,Kapaa,US,78.78,clear sky,22.0752,-159.3190,
14,Atuona,PF,77.61,light rain,-9.8000,-139.0333,
...,...,...,...,...,...,...,...
687,Palompon,PH,82.00,overcast clouds,11.0508,124.3843,
688,Aden,YE,75.81,clear sky,12.7794,45.0367,
690,Ventanas,EC,76.05,moderate rain,-1.4500,-79.4667,
693,Vao,NC,80.46,overcast clouds,-22.6667,167.4833,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        clean_hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,81.05,clear sky,-33.6500,115.3333,Observatory Guest House
3,Beira,MZ,75.81,few clouds,-19.8436,34.8389,Beira Terrace Hotel
5,Avarua,CK,75.25,light rain,-21.2078,-159.7750,Paradise Inn
12,Kapaa,US,78.78,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
14,Atuona,PF,77.61,light rain,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
686,Ugoofaaru,MV,80.96,overcast clouds,5.6667,73.0000,Ungoofaaru Inn
687,Palompon,PH,82.00,overcast clouds,11.0508,124.3843,Claudio A. Chiong Bed & Bath
688,Aden,YE,75.81,clear sky,12.7794,45.0367,Sama Emirate Hotel
690,Ventanas,EC,76.05,moderate rain,-1.4500,-79.4667,Hotel Kalabi INN


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 30.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))